In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_excel('C:/Users/user/OneDrive/Рабочий стол/Доп учеба/Хакатоны/ЦП 3 дашборд/train_dataset_rzhd_rating_train/dataset.xlsx')

In [4]:
def preprossesing_data(data_):
    data = data_.copy()
    data = data.dropna(how='all')
    data = data.drop(['Выполняемые функции', 'Должность за кем закреплен ТС'], axis=1).reset_index(drop=True)
    correct_ind = list(set(range(data.shape[0])) - set(data[data['Наименование полигона'].notna()].index))
    wrong_ind = list(data[data['Наименование полигона'].notna()].index)
    cols = ['Наименование полигона',
            'Краткое наименование',
            'Полигон',
            'Номерной знак ТС',
            'Наименование структурного подразделения',
            'Тип закрепления', 
            'манера вождения']
    for col in cols:
        data[col] = data[col].ffill()
    
    # Делаем нужный формат времени
    def replace_year(date):
        if date.year == 2019:
            return date.replace(year=2024)
        return date
    data['дата путевого листа'] = pd.to_datetime(data['дата путевого листа'].apply(replace_year), format='%d-%m-%Y')
    data['Дата сигнала телематики'] = pd.to_datetime(data['Дата сигнала телематики'].apply(replace_year), format='%d-%m-%Y')

    
    # делаем попарные даты
    wrong_ind += [data.shape[0]]
    info_date = pd.DataFrame()
    for i in range(len(wrong_ind) - 1):
        gr1 = data.loc[wrong_ind[i]+1:wrong_ind[i+1]-1, cols[:-2] + ['дата путевого листа', 'Данные путевых листов, пробег', 'манера вождения', 'Штрафы', 'Тип закрепления']].rename(columns={'дата путевого листа': 'Дата'})
        gr2 = data.loc[wrong_ind[i]+1:wrong_ind[i+1]-1, cols[:-2] + ['Дата сигнала телематики', 'Данные телематики, пробег', 'манера вождения', 'Штрафы', 'Тип закрепления']].rename(columns={'Дата сигнала телематики': 'Дата'})
        info_date = pd.concat([info_date, pd.merge(gr1, gr2, on=cols[:-2] + ['Дата', 'манера вождения', 'Тип закрепления'], how='outer').dropna(how='all')]).reset_index(drop=True)
        info_date = info_date.dropna(subset=['Дата'], how='all')

    info_date['Штрафы'] = info_date[['Штрафы_x', 'Штрафы_y']].fillna(0).max(axis=1)
    data = info_date.drop(columns=['Штрафы_x', 'Штрафы_y'])

    data['Поездка не по плану'] = data.apply(lambda row: 1 if pd.isna(row['Данные путевых листов, пробег']) and not pd.isna(row['Данные телематики, пробег']) else 0, axis=1)
    data['Не выполнил поездку'] = data.apply(lambda row: 1 if not pd.isna(row['Данные путевых листов, пробег']) and pd.isna(row['Данные телематики, пробег']) else 0, axis=1)

    return data


In [5]:
df = preprossesing_data(df)

In [6]:
df = df.sort_values(by='Дата', ascending=True)

In [7]:
df

,Наименование полигона,Краткое наименование,Полигон,Номерной знак ТС,Наименование структурного подразделения,Дата,"Данные путевых листов, пробег",манера вождения,Тип закрепления,"Данные телематики, пробег",Штрафы,Поездка не по плану,Не выполнил поездку
2356,ГОРЬКОВСКАЯ ЖД,ЦМ,Горьковская дир,Е560ХН150,ДМУ-3,2024-04-01,125.00,4.2,В целевой структуре парка,125.00,0.0,0,0
3186,ГОРЬКОВСКАЯ ЖД,ЦМ,Горьковская дир,АУ033966,ДМУ-2,2024-04-01,NaN,5.9,В целевой структуре парка,63.00,0.0,1,0
2215,ГОРЬКОВСКАЯ ЖД,ЦМ,Горьковская дир,3000НО50,ДМУ-4,2024-04-01,NaN,6.0,В целевой структуре парка,280.00,0.0,1,0
2248,ГОРЬКОВСКАЯ ЖД,ЦМ,Горьковская дир,3003НО50,ДМУ-4,2024-04-01,NaN,5.9,В целевой структуре парка,286.00,0.0,1,0
3167,ГОРЬКОВСКАЯ ЖД,ЦМ,Горьковская дир,Р956КК150,ДМУ-2,2024-04-01,63.00,5.9,В целевой структуре парка,NaN,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,МОСКОВСКАЯ ЖД,ЦМ,Московская ДМ,6909ОН57,Орловская механизированная дистанция погрузочн...,2024-04-30,NaN,5.9,В целевой структуре парка,82.62,0.0,1,0
949,МОСКОВСКАЯ ЖД,ЦМ,Московская ДМ,9440ВУ77,Московская механизированная дистанция погрузоч...,2024-04-30,1.00,5.4,В целевой структуре парка,1.00,3.0,0,0
3831,СЕВЕРНАЯ ЖД,ЦМ,Северная ДМ,С156ХУ07,Архангельская механизированная дистанция ПРР и КО,2024-04-30,432.00,5.9,В целевой структуре парка,NaN,0.0,0,1
3303,СЕВЕРНАЯ ЖД,ЦМ,Северная ДМ,М990УО07,Архангельская механизированная дистанция ПРР и КО,2024-04-30,208.00,5.8,В целевой структуре парка,208.00,0.0,0,0


In [8]:
df_filtered_poezdki = df[(df['Данные путевых листов, пробег'].notna()) & (df['Данные путевых листов, пробег'] != 0)]
df_filtered_telematika = df[(df['Данные телематики, пробег'].notna()) & (df['Данные телематики, пробег'] != 0)]
result_poezdki = df_filtered_poezdki.groupby(['Дата', 'Наименование структурного подразделения', 'Полигон'])['Данные путевых листов, пробег'].count().reset_index()
result_telematika = df_filtered_telematika.groupby(['Дата', 'Наименование структурного подразделения', 'Полигон'])['Данные телематики, пробег'].count().reset_index()
result = pd.merge(result_poezdki, result_telematika, on=['Дата', 'Наименование структурного подразделения', 'Полигон'], how='outer')
type_counts = df.groupby(['Дата', 'Наименование структурного подразделения', 'Полигон', 'Тип закрепления']).size().unstack(fill_value=0).reset_index()
result = pd.merge(result, type_counts, on=['Дата', 'Наименование структурного подразделения', 'Полигон'], how='outer')

In [9]:
result.head(15)

,Дата,Наименование структурного подразделения,Полигон,"Данные путевых листов, пробег","Данные телематики, пробег",В целевой структуре парка,Прочие
0,2024-04-01,Архангельская механизированная дистанция ПРР и КО,Северная ДМ,16.0,16.0,21,0
1,2024-04-01,Брянско-Смоленская механизированная дистанция ...,Московская ДМ,16.0,13.0,15,2
2,2024-04-01,ГТС Выборг,Окт ДМ,20.0,19.0,21,2
3,2024-04-01,ДМУ-2,Горьковская дир,11.0,17.0,21,2
4,2024-04-01,ДМУ-3,Горьковская дир,6.0,7.0,9,0
5,2024-04-01,ДМУ-4,Горьковская дир,8.0,12.0,15,0
6,2024-04-01,Калининградская ДМ,Клнг. ДУТСК,5.0,7.0,0,8
7,2024-04-01,Котласская механизированная дистанция ПРР и КО,Северная ДМ,5.0,3.0,4,1
8,2024-04-01,МЧ-01 г.Калининград,Клнг. ДУТСК,7.0,1.0,7,1
9,2024-04-01,Московская механизированная дистанция ОКТ ДМ (...,Окт ДМ,6.0,8.0,7,1


In [10]:
final_result = pd.DataFrame(result)

result['Номера в целевой структуре парка'] = [[] for _ in range(len(result))] # в колонке  Тип закрепления имеют значение В целевой структуре парка, при этом  Данные телематики, пробег значение 0 или Null(отдыхали)
result['Номерные знаки с пробегом'] = [[] for _ in range(len(result))] # в колонке Данные телематики, пробег	имеет значение не 0 и не Null
result['Прочие номера'] = [[] for _ in range(len(result))] # Тип закрепления имеет значение Прочие
result['Номера машин, которые целевые, но ездили, хотя не должны'] = [[] for _ in range(len(result))]

for idx, row in result.iterrows():
    df_filtered = df[
        (df['Дата'] == row['Дата']) &
        (df['Наименование структурного подразделения'] == row['Наименование структурного подразделения']) &
        (df['Полигон'] == row['Полигон'])
    ]
    should_not_have_moved = df_filtered[
        (df_filtered['Тип закрепления'] == 'В целевой структуре парка') &
        ((df_filtered['Данные телематики, пробег'] != 0) & (~df_filtered['Данные телематики, пробег'].isna())) &
        ((df_filtered['Данные путевых листов, пробег'] == 0) | (df_filtered['Данные путевых листов, пробег'].isna()))
    ]['Номерной знак ТС'].tolist()

    in_target_structure = df_filtered[
        (df_filtered['Тип закрепления'] == 'В целевой структуре парка') &
        ((df_filtered['Данные телематики, пробег'] == 0) | (df_filtered['Данные телематики, пробег'].isna()))
    ]['Номерной знак ТС'].tolist()
    
    with_telematics_mileage = df_filtered[
        (df_filtered['Данные телематики, пробег'] != 0) & (~df_filtered['Данные телематики, пробег'].isna())
    ]['Номерной знак ТС'].tolist()
    
    others = df_filtered[
        (df_filtered['Тип закрепления'] == 'Прочие')
    ]['Номерной знак ТС'].tolist()
    
    result.at[idx, 'Номера машин, которые целевые, но ездили, хотя не должны'] = should_not_have_moved
    result.at[idx, 'Номера в целевой структуре парка'] = in_target_structure
    result.at[idx, 'Номерные знаки с пробегом'] = with_telematics_mileage
    result.at[idx, 'Прочие номера'] = others
result

,Дата,Наименование структурного подразделения,Полигон,"Данные путевых листов, пробег","Данные телематики, пробег",В целевой структуре парка,Прочие,Номера в целевой структуре парка,Номерные знаки с пробегом,Прочие номера,"Номера машин, которые целевые, но ездили, хотя не должны"
0,2024-04-01,Архангельская механизированная дистанция ПРР и КО,Северная ДМ,16.0,16.0,21,0,"[С607ХУ07, О644НМ07, С156ХУ07, Р615ХН07, Р059Р...","[0701ХР76, Т160КН76, 3730АВ09, У415РМ07, О964Н...",[],"[0701ХР76, 3730АВ09, 0590ЕА30, 9369РХ61, 0003В..."
1,2024-04-01,Брянско-Смоленская механизированная дистанция ...,Московская ДМ,16.0,13.0,15,2,"[Б/Н, 0034СР67, 5590НХ30, 9444ВУ77]","[0407РК77, 4069НТ77, 4074НТ77, 7049СН67, О490М...","[Х605СР97, К331СР777]",[]
2,2024-04-01,ГТС Выборг,Окт ДМ,20.0,19.0,21,2,"[0316РВ70, 6346ТА69, 0061РХ70]","[6909РВ70, 0060РХ70, 0060РХ70, 0060РХ70, 0060Р...","[0675РХ70, 0693РХ70]","[0690РХ70, 4070РВ70]"
3,2024-04-01,ДМУ-2,Горьковская дир,11.0,17.0,21,2,"[Р956КК150, 5605УА01, ЕК706650, С667НТ150, С67...","[АУ033966, Н793КХ150, Х005ВТ50, Н659КВ150, 300...","[Х005ВТ50, 9164НК50]","[АУ033966, Н659КВ150, 3005НО50, 1044НЕ50, 7079..."
4,2024-04-01,ДМУ-3,Горьковская дир,6.0,7.0,9,0,"[С011ХУ150, М719АХ150]","[Е560ХН150, 7774НУ50, 3740МО43, 0003КЕ43, 3739...",[],"[3740МО43, 3739МО43, 3031НО50]"
...,...,...,...,...,...,...,...,...,...,...,...
352,2024-04-30,Орловская механизированная дистанция погрузочн...,Московская ДМ,5.0,7.0,10,0,"[Т033ТТ197, М059ОВ177, 7011ОУ57]","[М100СН177, 6930ОН57, А003АР57, Х905ХО197, 957...",[],"[6930ОН57, Х905ХО197, 9570ВС77, 0470ОН57, 6909..."
353,2024-04-13,ДМУ-3,Горьковская дир,NaN,1.0,1,0,[],[9376УА43],[],[9376УА43]
354,2024-04-21,ДМУ-3,Горьковская дир,NaN,1.0,1,0,[],[9376УА43],[],[9376УА43]
355,2024-04-26,Калининградская ДМ,Клнг. ДУТСК,NaN,3.0,0,3,[],"[7503КН39, 0573КН39, 7500КН39]","[7503КН39, 0573КН39, 7500КН39]",[]


In [11]:
result['lost_money'] = (result['Данные путевых листов, пробег'] - result['Данные телематики, пробег']).clip(lower=0)
poligon_lost_money = result.groupby('Полигон').agg({'lost_money': 'sum'})
poligon_lost_money

,lost_money
Полигон,
Горьковская дир,32.0
Клнг. ДУТСК,64.0
Московская ДМ,131.0
Окт ДМ,23.0
Северная ДМ,164.0


In [12]:
def give_chillusha(df_, start_date, end_date, group_by_column='Полигон', aggregate_column='Номера в целевой структуре парка'):
    df = df_.copy()
    df = df[(df['Дата'] >= start_date) & (df['Дата'] <= end_date)]
    poligon_car_relax = (
        df
        .explode(aggregate_column)
        .groupby(group_by_column)
        .agg({aggregate_column: list})
        .reset_index()
    )
    poligon_car_relax[aggregate_column] = poligon_car_relax[aggregate_column].apply(lambda x: list(filter(None, x)))


    return poligon_car_relax

In [14]:
give_chillusha(result, '2024-04-01', '2024-04-30', group_by_column='Наименование структурного подразделения', aggregate_column='Номера в целевой структуре парка')

,Наименование структурного подразделения,Номера в целевой структуре парка
0,Архангельская механизированная дистанция ПРР и КО,"[С607ХУ07, О644НМ07, С156ХУ07, Р615ХН07, Р059Р..."
1,Брянско-Смоленская механизированная дистанция ...,"[Б/Н, 0034СР67, 5590НХ30, 9444ВУ77, Б/Н, Б/Н, ..."
2,ГТС Выборг,"[0316РВ70, 6346ТА69, 0061РХ70, 7350ОН57, 0317Р..."
3,ДМУ-2,"[Р956КК150, 5605УА01, ЕК706650, С667НТ150, С67..."
4,ДМУ-3,"[С011ХУ150, М719АХ150, nan, 0050НО50, С011ХУ15..."
5,ДМУ-4,"[С791ХУ150, 3075НО50, С790ХУ150, nan, 3075НО50..."
6,Калининградская ДМ,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
7,Котласская механизированная дистанция ПРР и КО,"[М014МО07, nan, М014МО07, У375ТВ07, М014МО07, ..."
8,МЧ-01 г.Калининград,"[0090АВ39, 0030КО39, Н075ХТ39, 4960АМ39, 4961А..."
9,Московская механизированная дистанция ОКТ ДМ (...,"[nan, nan, nan, А593АС761, 3344МР51, nan, nan,..."


In [98]:
# возвращает машины, которые целевые, но никуда не ездили за определенную дату на уровне полигона
def give_chillusha(df_, date):
    df = df_.copy()
    df = df[df['Дата'] == date]
    poligon_car_relax = (
        df
        .explode('Номера в целевой структуре парка')
        .groupby('Полигон')
        .agg({'Номера в целевой структуре парка': list})
        .reset_index()
    )
    return poligon_car_relax

In [106]:
# возвращает машины, которые дргуие
def give_others(df_, date):
    df = df_.copy()
    df = df[df['Дата'] == date]
    poligon_car_others = (
        df
        .explode('Прочие номера')
        .groupby('Полигон')
        .agg({'Прочие номера': list})
        .reset_index()
    )
    return poligon_car_others

In [107]:
give_others(result, '2024-04-01')

,Полигон,Прочие номера
0,Горьковская дир,"[Х005ВТ50, 9164НК50, nan, nan]"
1,Клнг. ДУТСК,"[0905КН39, 0914КН39, 0904КН39, 0907КН39, 7503К..."
2,Московская ДМ,"[Х605СР97, К331СР777, А094ОХ777, 0536РК77, Н00..."
3,Окт ДМ,"[0693РХ70, 0675РХ70, Т906НТ170]"
4,Северная ДМ,"[nan, В013ЕТ174]"


In [108]:
give_chillusha(result, '2024-04-01')

,Полигон,Номера в целевой структуре парка
0,Горьковская дир,"[С677ЕЕ150, С549СМ150, Р956КК150, С667НТ150, Е..."
1,Клнг. ДУТСК,"[nan, 0030КО39, 0090АВ39, Н075ХТ39, 4960АМ39, ..."
2,Московская ДМ,"[0034СР67, Б/Н, 5590НХ30, 9444ВУ77, 9400ВУ77, ..."
3,Окт ДМ,"[0061РХ70, 6346ТА69, 0316РВ70, nan]"
4,Северная ДМ,"[Р059РВ07, С156ХУ07, Р615ХН07, С607ХУ07, О644Н..."


In [76]:
poligon_car_relax = (
    result
    .explode('Номера в целевой структуре парка')
    .groupby('Полигон')
    .agg({'Номера в целевой структуре парка': list})
    .reset_index()
)
poligon_car_relax

,Полигон,Номера в целевой структуре парка
0,Горьковская дир,"[С677ЕЕ150, С549СМ150, Р956КК150, С667НТ150, Е..."
1,Клнг. ДУТСК,"[nan, 0030КО39, 0090АВ39, Н075ХТ39, 4960АМ39, ..."
2,Московская ДМ,"[0034СР67, Б/Н, 5590НХ30, 9444ВУ77, 9400ВУ77, ..."
3,Окт ДМ,"[0061РХ70, 6346ТА69, 0316РВ70, nan, 0317РВ70, ..."
4,Северная ДМ,"[Р059РВ07, С156ХУ07, Р615ХН07, С607ХУ07, О644Н..."


In [39]:
poligon_podrazdel = df.groupby('Полигон')['Наименование структурного подразделения'].unique().reset_index()
poligon_podrazdel

,Полигон,Наименование структурного подразделения
0,Горьковская дир,"[ДМУ-3, ДМУ-2, ДМУ-4]"
1,Клнг. ДУТСК,"[Калининградская ДМ, МЧ-01 г.Калининград]"
2,Московская ДМ,[Брянско-Смоленская механизированная дистанция...
3,Окт ДМ,[Московская механизированная дистанция ОКТ ДМ ...
4,Северная ДМ,[Архангельская механизированная дистанция ПРР ...


In [138]:
result.iloc[3]['Номера в целевой структуре парка']

['С677ЕЕ150', 'С549СМ150', 'Р956КК150', 'С667НТ150', 'ЕК706650', '5605УА01']

In [134]:
result.iloc[3]['Номерные знаки с пробегом']

['3005НО50',
 'С669НТ150',
 'ЕК706550',
 'АУ033966',
 'Н793КХ150',
 'Х005ВТ50',
 'Н659КВ150',
 '7079НО50',
 '7000АА50',
 '4010АА50',
 '7411АА50',
 '7197НР50',
 '9164НК50',
 '1044НЕ50',
 '0594НЕ50',
 'Б/Н',
 '0051ТМ71']

In [135]:
result.iloc[3]['Прочие номера']

['Х005ВТ50', '9164НК50']

In [136]:
result.iloc[3]['Номера машин, которые целевые, но ездили, хотя не должны']

['3005НО50',
 'ЕК706550',
 'АУ033966',
 'Н659КВ150',
 '7079НО50',
 '7000АА50',
 '7411АА50',
 '7197НР50',
 '1044НЕ50',
 '0594НЕ50',
 '0051ТМ71']